### Investigating American Degrees

In this notebook, we take a look at https://collegescorecard.ed.gov/data/ and the Most Recent Data by Field of Study (as of Oct 2020).

## Data Cleaning

# imports
import os
import re
import numpy as np
import pandas as pd

In [62]:
df = pd.read_csv("college-fields-of-study.csv")
df.head()

,UNITID,OPEID6,INSTNM,CONTROL,MAIN,CIPCODE,CIPDESC,CREDLEV,CREDDESC,COUNT,DEBTMEDIAN,DEBTPAYMENT10YR,DEBTMEAN,TITLEIVCOUNT,EARNINGSCOUNT,MD_EARN_WNE,IPEDSCOUNT1,IPEDSCOUNT2
0,100654.0,1002,Alabama A & M University,Public,1,100,"Agriculture, General.",3,Bachelors Degree,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,4.0,NaN
1,100654.0,1002,Alabama A & M University,Public,1,101,Agricultural Business and Management.,3,Bachelors Degree,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,NaN,NaN
2,100654.0,1002,Alabama A & M University,Public,1,109,Animal Sciences.,3,Bachelors Degree,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,31066,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,7.0,6.0
3,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,3,Bachelors Degree,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,5.0,8.0
4,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,5,Master's Degree,15,PrivacySuppressed,PrivacySuppressed,61527,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,9.0,10.0


In [63]:
print(df.shape)

(216638, 18)


In [64]:
new_df = df[df['EARNINGSCOUNT'] != "PrivacySuppressed"]
new_df = new_df[new_df['COUNT'] != "PrivacySuppressed"] ## Remove privacy suppressed rows in these columns
new_df.dropna() ## Remove rows with NULL values
new_df.head() 

,UNITID,OPEID6,INSTNM,CONTROL,MAIN,CIPCODE,CIPDESC,CREDLEV,CREDDESC,COUNT,DEBTMEDIAN,DEBTPAYMENT10YR,DEBTMEAN,TITLEIVCOUNT,EARNINGSCOUNT,MD_EARN_WNE,IPEDSCOUNT1,IPEDSCOUNT2
15,100654.0,1002,Alabama A & M University,Public,1,1101,"Computer and Information Sciences, General.",3,Bachelors Degree,25,36250,376,37194,33,33,54300,17.0,16.0
24,100654.0,1002,Alabama A & M University,Public,1,1313,Teacher Education and Professional Development...,3,Bachelors Degree,46,31505,327,33085,32,31,25400,24.0,33.0
27,100654.0,1002,Alabama A & M University,Public,1,1410,"Electrical, Electronics and Communications Eng...",3,Bachelors Degree,22,34809,361,36482,33,31,54500,19.0,11.0
37,100654.0,1002,Alabama A & M University,Public,1,2401,"Liberal Arts and Sciences, General Studies and...",3,Bachelors Degree,39,29956,311,30970,30,30,24000,28.0,28.0
38,100654.0,1002,Alabama A & M University,Public,1,2601,"Biology, General.",3,Bachelors Degree,59,34000,353,32783,31,30,23800,30.0,43.0


Unfortunately, we lose a lot of data after cleaning out NULL values and any rows that contain "PrivacySuppressed". This string indicates that a 

In [65]:
new_df['DEBTMEDIAN'].str.contains('PrivacySuppressed').any()

True

In [66]:
new_df = new_df[new_df['DEBTMEDIAN'] != "PrivacySuppressed"]

In [67]:
# new_df['DEBTPAYMENT10YR'].str.contains('PrivacySuppressed').any()
# new_df['DEBTMEAN'].str.contains('PrivacySuppressed').any()
# new_df['TITLEIVCOUNT'].str.contains('PrivacySuppressed').any()
# new_df['EARNINGSCOUNT'].str.contains('PrivacySuppressed').any()
new_df['MD_EARN_WNE'].str.contains('PrivacySuppressed').any() ## Checking to see if any of the columns still contain suppressed data

False

In [68]:
print(new_df.shape)
new_df.head() ## This should be the final dataset after cleaning

(36720, 18)


,UNITID,OPEID6,INSTNM,CONTROL,MAIN,CIPCODE,CIPDESC,CREDLEV,CREDDESC,COUNT,DEBTMEDIAN,DEBTPAYMENT10YR,DEBTMEAN,TITLEIVCOUNT,EARNINGSCOUNT,MD_EARN_WNE,IPEDSCOUNT1,IPEDSCOUNT2
15,100654.0,1002,Alabama A & M University,Public,1,1101,"Computer and Information Sciences, General.",3,Bachelors Degree,25,36250,376,37194,33,33,54300,17.0,16.0
24,100654.0,1002,Alabama A & M University,Public,1,1313,Teacher Education and Professional Development...,3,Bachelors Degree,46,31505,327,33085,32,31,25400,24.0,33.0
27,100654.0,1002,Alabama A & M University,Public,1,1410,"Electrical, Electronics and Communications Eng...",3,Bachelors Degree,22,34809,361,36482,33,31,54500,19.0,11.0
37,100654.0,1002,Alabama A & M University,Public,1,2401,"Liberal Arts and Sciences, General Studies and...",3,Bachelors Degree,39,29956,311,30970,30,30,24000,28.0,28.0
38,100654.0,1002,Alabama A & M University,Public,1,2601,"Biology, General.",3,Bachelors Degree,59,34000,353,32783,31,30,23800,30.0,43.0


## Some Major Analysis

Here we will look at avg earnings across majors, obviously the major names are varied so we may see a lot of different buckets. First let's see differences between Bachelor's, Master's and Doctorate holders.

The column "MD_EARN_WNE" represents the earnings by field of study one year after degree completion. Here, we look at how much a graduate of a given degree type earns on average a year after completing their degree.

In [75]:
## Bachelors
bach_df = new_df[new_df['CREDDESC'] == 'Bachelors Degree']
np.asarray(bach_df["MD_EARN_WNE"], dtype=np.float).mean() ## Convert to a numpy array without data issues

38509.036113543865

In [76]:
print(bach_df.shape)

(19411, 18)


In [81]:
## Masters
mast_df = new_df[new_df['CREDDESC'] == "Master's Degree"]
np.asarray(mast_df["MD_EARN_WNE"], dtype=np.float).mean()

56367.92097836312

In [82]:
print(mast_df.shape)

(6378, 18)


In [83]:
## Doctorate Degrees
doct_df = new_df[new_df['CREDDESC'] == "Doctoral Degree"]
np.asarray(doct_df["MD_EARN_WNE"], dtype=np.float).mean()

72919.35483870968

In [84]:
print(doct_df.shape)

(279, 18)
